In [19]:
import pandas as pd

# Load the original CSV file containing raw data
file_path = "Combined_Clean_1.csv"
df = pd.read_csv(file_path)

# Convert 'publish_date' to datetime and extract only the date for consistent date handling
df['publish_date'] = pd.to_datetime(df['publish_date']).dt.date

# Rename columns to more descriptive and meaningful names for clarity
df.rename(columns={
    'average_sentiment': 'article_sentiment',
    'daily_average_sentiment': 'average_market_sentiment',
    'Average_Market_Change': 'average_market_percent_change',
    'Day_Percent_Change': 'Yesterday_Percent_Change'
}, inplace=True)

# Define sentiment classification function to categorize sentiment scores
# This helps in quickly understanding the sentiment polarity
def classify_sentiment(value):
    if value >= 0.4:
        return 'Bullish'
    elif -0.2 <= value < 0.4:
        return 'Neutral'
    else:
        return 'Bearish'

# Add categorical sentiment columns for easier interpretation of sentiment scores
df['article_sentiment_class'] = df['article_sentiment'].apply(classify_sentiment)
df['average_market_sentiment_class'] = df['average_market_sentiment'].apply(classify_sentiment)

# Define columns to be aggregated, focusing on numerical sentiment and performance metrics
columns_to_average = [
    'article_sentiment',
    'average_market_sentiment',
    'average_market_percent_change',
    'RatingScore_pct_change',
    'analyst_score_pct_change',
    'target_median_price_pct_change',
    'target_score',
    'Yesterday_Percent_Change'
]

# Group data by publish date and ticker, calculating mean values to reduce noise and get daily insights
daily_ticker_aggregated_df = df.groupby(['publish_date', 'ticker'])[columns_to_average].mean().reset_index()

# Sort the dataframe to ensure correct time-series calculations
daily_ticker_aggregated_df.sort_values(['ticker', 'publish_date'], inplace=True)

# Calculate a smoothed market sentiment by incorporating previous day's sentiment
# This helps in capturing sentiment trend rather than daily fluctuations
daily_ticker_aggregated_df['previous_market_sentiment'] = daily_ticker_aggregated_df.groupby('ticker')['average_market_sentiment'].shift(1)
daily_ticker_aggregated_df['average_market_sentiment'] = (daily_ticker_aggregated_df['average_market_sentiment'] + daily_ticker_aggregated_df['previous_market_sentiment']) / 2
daily_ticker_aggregated_df.drop(columns=['previous_market_sentiment'], inplace=True)

# Add Next_Day_Percent_Change column to enable forward-looking analysis
daily_ticker_aggregated_df['Next_Day_Percent_Change'] = daily_ticker_aggregated_df.groupby('ticker')['Yesterday_Percent_Change'].shift(-1)

# Reclassify sentiment for the aggregated data
daily_ticker_aggregated_df['article_sentiment_class'] = daily_ticker_aggregated_df['article_sentiment'].apply(classify_sentiment)
daily_ticker_aggregated_df['average_market_sentiment_class'] = daily_ticker_aggregated_df['average_market_sentiment'].apply(classify_sentiment)

# Remove any rows with blank/NaN cells to ensure data integrity
daily_ticker_aggregated_df.dropna(inplace=True)

# Save the processed and aggregated DataFrame to a new CSV file
output_file_path = "Daily_Ticker_Table.csv"
daily_ticker_aggregated_df.to_csv(output_file_path, index=False)

# Inform the user about successful file creation
print(f"New CSV file with daily averages grouped by ticker saved as {output_file_path}")

















New CSV file with daily averages grouped by ticker saved as Daily_Ticker_Table.csv


In [57]:
import pandas as pd
import re

# Load the CSV file
file_path = "Combined_Raw.csv"
df = pd.read_csv(file_path)

# Filter rows where word_count > 7
# Reason: To focus only on rows with meaningful content (articles or entries with sufficient length).
filtered_df = df[df['word_count'] > 7]

# Function to check if a string contains only ASCII characters
# Reason: Non-ASCII characters can cause encoding issues during processing.
def is_ascii(text):
    try:
        return text.encode('ascii').decode() == text
    except UnicodeEncodeError:
        return False

# Filter rows where the title column contains only ASCII characters
# Reason: Ensures compatibility with ASCII-based text processing or analysis.
if 'title' in filtered_df.columns:
    filtered_df = filtered_df[filtered_df['title'].apply(is_ascii)]

# Drop specific columns that are unnecessary or irrelevant for the analysis
# Reason: These columns do not contribute to the analysis and are removed to streamline the dataset.
columns_to_drop = [
    'Strong_Buy', 'Buy', 'Hold', 'Sell', 'Strong_Sell',
    'hourly_date', 'date_only', 'week_of_year'
]
filtered_df = filtered_df.drop(columns=columns_to_drop, errors='ignore')

# Ensure the publish_date column is in datetime format
# Reason: Proper datetime format is required for sorting and time-based analysis.
filtered_df['publish_date'] = pd.to_datetime(filtered_df['publish_date'], errors='coerce')

# Sort the data by ticker and publish_date
# Reason: Sorting ensures chronological order within each ticker, necessary for forward filling and time-series calculations.
filtered_df = filtered_df.sort_values(by=['ticker', 'publish_date'])

# Forward fill missing values for selected columns within each ticker group
# Reason: Missing values may occur in financial or analyst data. Forward filling propagates the most recent values to fill gaps.
fill_columns = [
    'RatingScore', 
    'analyst_score', 
    'reward_score', 
    'risk_score', 
    'target_score', 
    'target_median_price',
    'Close',
    'Volume',
    'High',
    'Low',
    'Open'
]
fill_columns = [col for col in fill_columns if col in filtered_df.columns]

if fill_columns:
    filtered_df[fill_columns] = filtered_df.groupby('ticker')[fill_columns].fillna(method='ffill')

# Calculate Day_Percent_Change for market data
# Reason: Provides a quick measure of daily price movement as a percentage change from the opening to the closing price.
filtered_df['Day_Percent_Change'] = ((filtered_df['Close'] - filtered_df['Open']) / filtered_df['Open'] * 100).round(2)

# Calculate raw differences for Forward_*min_Change columns relative to Percent_Difference
# Reason: Quantifies how the short-term forward changes compare to the overall percent difference.
forward_columns = [
    'Forward_15min_Change',
    'Forward_30min_Change',
    'Forward_45min_Change',
    'Forward_60min_Change'
]

if 'Percent_Difference' in filtered_df.columns:
    for column in forward_columns:
        if column in filtered_df.columns:
            new_column_name = f'{column}_Diff'
            filtered_df[new_column_name] = (filtered_df[column] - filtered_df['Percent_Difference'])

# Calculate daily percent changes for specified columns
# Reason: Adds day-over-day percentage changes to analyze trends in financial or analyst data.
pct_change_columns = [
    'RatingScore',
    'analyst_score',
    'target_score',
    'target_median_price'
]

def calculate_daily_pct_change(group, column):
    # Get the first value of the column for each date
    daily_values = group.groupby(group['publish_date'].dt.date)[column].first()
    # Calculate percentage change between consecutive days
    daily_pct_change = daily_values.pct_change().mul(100).round(2)
    # Map these changes back to all rows for each date
    return group['publish_date'].dt.date.map(daily_pct_change)

for column in pct_change_columns:
    if column in filtered_df.columns:
        new_column = f'{column}_pct_change'
        filtered_df[new_column] = filtered_df.groupby('ticker').apply(
            lambda x: calculate_daily_pct_change(x, column)
        ).reset_index(level=0, drop=True)

# Calculate the average sentiment and add it as a new column
# Reason: Combines multiple sentiment scores into a single metric for simplicity.
filtered_df['average_sentiment'] = filtered_df[['textblob_sentiment', 'vader_sentiment']].mean(axis=1)

# Calculate the daily average sentiment
# Reason: Aggregates sentiment scores on a daily basis to analyze overall market sentiment trends.
daily_avg_sentiment = filtered_df.groupby(filtered_df['publish_date'].dt.date)['average_sentiment'].mean().reset_index()
daily_avg_sentiment.rename(columns={'average_sentiment': 'daily_average_sentiment', 'publish_date': 'publish_date'}, inplace=True)

# Merge the daily average sentiment back into the original DataFrame
# Reason: Links the aggregated daily sentiment scores to the detailed dataset for analysis.
filtered_df['publish_date_date'] = filtered_df['publish_date'].dt.date
filtered_df = filtered_df.merge(daily_avg_sentiment, left_on='publish_date_date', right_on='publish_date', how='left')

# Remove unnecessary columns introduced during the merge
# Reason: Clean up temporary columns to maintain dataset integrity and readability.
filtered_df.drop(columns=['publish_date_y'], inplace=True, errors='ignore')
filtered_df.rename(columns={'publish_date_x': 'publish_date'}, inplace=True, errors='ignore')

# Calculate the daily average market change
# Reason: Provides a metric for average market performance across all entries on a given day.
daily_avg_change = filtered_df.groupby(filtered_df['publish_date_date'])['Percent_Difference'].mean().reset_index()
daily_avg_change.rename(columns={'Percent_Difference': 'Average_Market_Change', 'publish_date_date': 'publish_date_date'}, inplace=True)

# Merge the daily average market change back into the original DataFrame
# Reason: Adds the aggregated market performance metric to the detailed dataset.
filtered_df = filtered_df.merge(daily_avg_change, left_on='publish_date_date', right_on='publish_date_date', how='left')

# Drop the temporary date column
# Reason: Temporary columns are no longer needed after merging the aggregated data.
filtered_df.drop(columns=['publish_date_date'], inplace=True)

# Save the resulting dataset
# Reason: Outputs the final cleaned and enriched dataset for further analysis or reporting.
output_file_path = "Combined_Clean_1.csv"
filtered_df.to_csv(output_file_path, index=False)

print(f"Filtered dataset with all calculations saved to {output_file_path}")













C:\Users\Bryce\AppData\Local\Temp\ipykernel_24388\1065970316.py:59: FutureWarning: DataFrameGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use DataFrame.fillna instead
  filtered_df[fill_columns] = filtered_df.groupby('ticker')[fill_columns].fillna(method='ffill')
C:\Users\Bryce\AppData\Local\Temp\ipykernel_24388\1065970316.py:59: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  filtered_df[fill_columns] = filtered_df.groupby('ticker')[fill_columns].fillna(method='ffill')
C:\Users\Bryce\AppData\Local\Temp\ipykernel_24388\1065970316.py:93: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fil

Filtered dataset with all calculations saved to Combined_Clean_1.csv


In [22]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from textblob import TextBlob
from newspaper import Article
import nltk
import os

import nltk
nltk.download('punkt')


def fetch_article_summary(link):
    try:
        article = Article(link)
        article.download()
        article.parse()
        article.nlp()
        return article.summary
    except Exception as e:
        print(f"[ERROR] Failed to fetch or summarize article: {e}")
        return "No summary available."

def calculate_sentiment(text):
    try:
        analysis = TextBlob(text)
        return analysis.sentiment.polarity
    except Exception as e:
        print(f"[ERROR] Sentiment analysis failed: {e}")
        return 0

def label_sentiment(score):
    if score > 0.35:
        return "Bullish"
    elif 0.15 < score <= 0.35:
        return "Somewhat-Bullish"
    elif -0.15 <= score <= 0.15:
        return "Neutral"
    elif -0.35 <= score < -0.15:
        return "Somewhat-Bearish"
    else:
        return "Bearish"

def get_market_news(tickers):
    all_news = []
    today = datetime.now().date()
    one_day_ago = today - timedelta(days=1)

    for ticker in tickers:
        stock = yf.Ticker(ticker)
        try:
            news = stock.news
            for item in news:
                try:
                    publish_timestamp = item.get('providerPublishTime', 0)
                    publish_date = datetime.fromtimestamp(publish_timestamp).date()

                    if publish_date >= one_day_ago and item.get('type', '').lower() == 'story':
                        link = item.get('link', '')
                        summary = fetch_article_summary(link) if link else "No summary available."
                        sentiment_score = calculate_sentiment(summary)
                        sentiment_label = label_sentiment(sentiment_score)

                        news_item = {
                            'ticker': ticker,
                            'title': item.get('title', ''),
                            'summary': summary,
                            'publisher': item.get('publisher', ''),
                            'link': link,
                            'publish_date': datetime.fromtimestamp(publish_timestamp),
                            'type': item.get('type', ''),
                            'related_tickers': ', '.join(item.get('relatedTickers', [])),
                            'source': 'yahoo',
                            'overall_sentiment_score': sentiment_score,
                            'overall_sentiment_label': sentiment_label,
                        }
                        all_news.append(news_item)
                except Exception as news_item_error:
                    print(f"[ERROR] Error processing news item: {news_item_error}")
        except Exception as e:
            print(f"[ERROR] Error retrieving news for {ticker}: {str(e)}")
    return pd.DataFrame(all_news)

def save_to_csv(df, filename):
    try:
        df.to_csv(filename, index=False)
        print(f"[INFO] Data successfully saved to {filename}")
    except Exception as e:
        print(f"[ERROR] Failed to save data to CSV: {e}")

def fetch_and_save_market_news():
    indices = ['^IXIC', '^DJI', '^RUT', '^GSPC']
    market_news = get_market_news(tickers=indices)
    if not market_news.empty:
        market_news['category'] = 'General'

    tech_stocks = [
        'AAPL', 'GOOGL', 'MSFT', 'ASTS', 'PTON', 'GSAT', 'PLTR', 'SMR', 'ACHR',
        'BWXT', 'ARBK', 'AMD', 'NVDA', 'BTC', 'GME', 'MU', 'TSLA', 'NFLX', 'ZG',
        'AVGO', 'SMCI', 'GLW', 'HAL', 'LMT', 'AMZN', 'CRM', 'NOW', 'CHTR', 'TDS', 'META','RGTI','QUBT',
        'LX', 'OKLO', 'PSIX', 'QFIN', 'RTX', 'TWLO'
    ]
    tech_news = get_market_news(tickers=tech_stocks)
    if not tech_news.empty:
        tech_news['category'] = 'Tech'

    combined_news = pd.concat([market_news, tech_news], ignore_index=True)

    if not combined_news.empty:
        save_to_csv(combined_news, "market_news.csv")
    else:
        print("[INFO] No news data to save.")

if __name__ == "__main__":
    fetch_and_save_market_news()










[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BryceDaniel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[ERROR] Failed to fetch or summarize article: 
**********************************************************************
  Resource punkt_tab not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt_tab')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt_tab/english/

  Searched in:
    - 'C:\\Users\\BryceDaniel/nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\BryceDaniel\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\OneDrive - Lincoln Telephone Company\\MSBA\\GitHub\\TrendSense\\nltk_data'
    - 'c:\\Users\\BryceDaniel\\OneDrive - Lincoln Telephone Compan

In [30]:
import http.client
import json
import csv
from datetime import datetime

def fetch_latest_yahoo_recommendations(ticker):
    print(f"Fetching recommendations for {ticker} from Yahoo Finance...")
    
    # Initialize connection
    conn = http.client.HTTPSConnection("yahoo-finance166.p.rapidapi.com")
    headers = {
        'x-rapidapi-key': "ee72be2ef9msh532c4fc1a7b7941p1176e1jsn0328598b0245",
        'x-rapidapi-host': "yahoo-finance166.p.rapidapi.com"
    }
    
    # API request
    conn.request("GET", f"/api/stock/get-recommendation-trend?symbol={ticker}&region=US", headers=headers)
    res = conn.getresponse()
    data = res.read()
    
    # Decode and parse response
    decoded_data = data.decode("utf-8")
    print("Raw Data:\n", decoded_data)  # Debugging: Print raw data
    parsed_data = json.loads(decoded_data)
    
    # Extract recommendations
    recommendations = parsed_data.get('quoteSummary', {}).get('result', [])[0].get('recommendationTrend', {}).get('trend', [])
    
    if recommendations:
        # Get the latest period's data
        latest_trend = recommendations[0]  # First entry is the latest
        latest_recommendation = {
            "Ticker": ticker,
            "Date": datetime.now().strftime('%Y-%m-%d'),
            "Period": latest_trend.get('period', 'N/A'),
            "Strong Buy": latest_trend.get('strongBuy', 0),
            "Buy": latest_trend.get('buy', 0),
            "Hold": latest_trend.get('hold', 0),
            "Sell": latest_trend.get('sell', 0),
            "Strong Sell": latest_trend.get('strongSell', 0)
        }
        return latest_recommendation
    else:
        print("No recommendation data found.")
        return None

def save_recommendations_to_csv(data, filename="latest_analyst_recommendations.csv"):
    print(f"Saving latest recommendations to {filename}...")
    with open(filename, mode="w", newline="") as file:
        fieldnames = ["Ticker", "Date", "Period", "Strong Buy", "Buy", "Hold", "Sell", "Strong Sell"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerow(data)
    print(f"Data saved to {filename}")

# Example usage
ticker = "AAPL"
latest_recommendation = fetch_latest_yahoo_recommendations(ticker)
if latest_recommendation:
    save_recommendations_to_csv(latest_recommendation)
else:
    print("No data available to save.")



Fetching recommendations for AAPL from Yahoo Finance...
Raw Data:
 {"quoteSummary":{"result":[{"recommendationTrend":{"trend":[{"period":"0m","strongBuy":8,"buy":24,"hold":12,"sell":1,"strongSell":2},{"period":"-1m","strongBuy":8,"buy":24,"hold":12,"sell":1,"strongSell":2},{"period":"-2m","strongBuy":8,"buy":23,"hold":12,"sell":1,"strongSell":2},{"period":"-3m","strongBuy":8,"buy":24,"hold":12,"sell":0,"strongSell":2}],"maxAge":86400}}],"error":null}}
Saving latest recommendations to latest_analyst_recommendations.csv...
Data saved to latest_analyst_recommendations.csv


In [ ]:
import os
import json
import logging
from typing import List, Dict, Any
from datetime import datetime

# Google Cloud libraries
from google.cloud import bigquery
import functions_framework
import requests

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize BigQuery client
client = bigquery.Client()

# Configuration
TABLE_ID = os.environ.get('BIGQUERY_TABLE', 'trendsense.stock_data.stock_analyst')
YAHOO_API_KEY = "ee72be2ef9msh532c4fc1a7b7941p1176e1jsn0328598b0245"
SEEKING_ALPHA_API_KEY ="ee72be2ef9msh532c4fc1a7b7941p1176e1jsn0328598b0245"

# Default tickers if not provided
DEFAULT_TICKERS = [
    {"ticker": "AAPL", "ticker_id": "146"},      # Apple
    {"ticker": "GOOGL", "ticker_id": "97"},      # Google
    {"ticker": "MSFT", "ticker_id": "152"},      # Microsoft
    {"ticker": "ASTS", "ticker_id": "12600"},    # AST SpaceMobile
    {"ticker": "PTON", "ticker_id": "484"},      # Peloton
    {"ticker": "GSAT", "ticker_id": "821"},      # Globalstar
    {"ticker": "PLTR", "ticker_id": "347"},      # Palantir
    {"ticker": "SMR", "ticker_id": "27514"},     # NuScale Energy
    {"ticker": "ACHR", "ticker_id": "8796"},     # Archer Aviation
    {"ticker": "BWXT", "ticker_id": "9323"},     # BWX Technologies
    {"ticker": "ARBK", "ticker_id": "34335"},    # Arb케k
    {"ticker": "AMD", "ticker_id": "202"},       # Advanced Micro Devices
    {"ticker": "NVDA", "ticker_id": "164"},      # NVIDIA
    {"ticker": "BTC", "ticker_id": "57628"},     # Bitcoin ETF
    {"ticker": "GME", "ticker_id": "364"},       # GameStop
    {"ticker": "MU", "ticker_id": "122"},        # Micron Technology
    {"ticker": "TSLA", "ticker_id": "383"},      # Tesla
    {"ticker": "NFLX", "ticker_id": "220"},      # Netflix
    {"ticker": "ZG", "ticker_id": "1307"},       # Zillow
    {"ticker": "AVGO", "ticker_id": "6509"},     # Broadcom
    {"ticker": "SMCI", "ticker_id": "11001"},    # Super Micro Computer
    {"ticker": "GLW", "ticker_id": "508"},       # Corning
    {"ticker": "HAL", "ticker_id": "82"},        # Halliburton
    {"ticker": "LMT", "ticker_id": "112"},       # Lockheed Martin
    {"ticker": "AMZN", "ticker_id": "1"},         # Amazon
    {"ticker": "CRM", "ticker_id": "231"},       # Salesforce
    {"ticker": "NOW", "ticker_id": "432"},       # ServiceNow
    {"ticker": "CHTR", "ticker_id": "230"},      # Charter Communications
    {"ticker": "TDS", "ticker_id": "7958"},      # Telephone and Data Systems
    {"ticker": "META", "ticker_id": "442"},      # Meta (Facebook)
    {"ticker": "RGTI", "ticker_id": "35854"}     # Rigetti Computing
]

def fetch_latest_yahoo_recommendations(ticker: str) -> Dict[str, Any] | None:
    """Fetch the latest analyst recommendations from Yahoo Finance."""
    logger.info(f"Fetching recommendations for {ticker} from Yahoo Finance...")
    
    if not YAHOO_API_KEY:
        logger.error("Yahoo Finance API key not configured")
        return None
    
    try:
        url = f"https://yahoo-finance166.p.rapidapi.com/api/stock/get-recommendation-trend"
        headers = {
            'x-rapidapi-key': YAHOO_API_KEY,
            'x-rapidapi-host': 'yahoo-finance166.p.rapidapi.com'
        }
        params = {
            'symbol': ticker,
            'region': 'US'
        }
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code != 200:
            logger.error(f"Yahoo API error: {response.status_code} {response.reason}")
            return None
        
        parsed_data = response.json()
        recommendations = parsed_data.get('quoteSummary', {}).get('result', [])[0].get('recommendationTrend', {}).get('trend', [])
        
        if recommendations:
            latest_trend = recommendations[0]
            return {
                "Ticker": ticker,
                "Date": datetime.now().strftime('%Y-%m-%d'),
                "Period": latest_trend.get('period', 'N/A'),
                "Strong_Buy": latest_trend.get('strongBuy', 0),
                "Buy": latest_trend.get('buy', 0),
                "Hold": latest_trend.get('hold', 0),
                "Sell": latest_trend.get('sell', 0),
                "Strong_Sell": latest_trend.get('strongSell', 0)
            }
        else:
            logger.warning("No recommendation data found in the response.")
    except Exception as e:
        logger.error(f"Error fetching Yahoo recommendations: {e}")
    return None

def fetch_seeking_alpha_targets(ticker_id: str) -> Dict[str, float] | None:
    """Fetch price target data from Seeking Alpha."""
    logger.info(f"Fetching price targets for Ticker ID {ticker_id} from Seeking Alpha...")
    
    if not SEEKING_ALPHA_API_KEY:
        logger.error("Seeking Alpha API key not configured")
        return None
    
    try:
        url = "https://seeking-alpha.p.rapidapi.com/symbols/get-analyst-price-target"
        headers = {
            'x-rapidapi-key': SEEKING_ALPHA_API_KEY,
            'x-rapidapi-host': 'seeking-alpha.p.rapidapi.com'
        }
        params = {
            'ticker_ids': ticker_id,
            'return_window': 1,
            'group_by_month': 'false'
        }
        
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code != 200:
            logger.error(f"Seeking Alpha API error: {response.status_code} {response.reason}")
            return None
        
        parsed_data = response.json()
        estimates = parsed_data.get('estimates', {}).get(str(ticker_id), {})
        
        return {
            "Target_Low": float(estimates.get("target_price_low", {}).get("0", [{}])[0].get("dataitemvalue", 0)),
            "Target_Mean": float(estimates.get("target_price", {}).get("0", [{}])[0].get("dataitemvalue", 0)),
            "Target_High": float(estimates.get("target_price_high", {}).get("0", [{}])[0].get("dataitemvalue", 0))
        }
    except Exception as e:
        logger.error(f"Error fetching Seeking Alpha price targets: {e}")
    return None

def create_or_get_table() -> bigquery.Table:
    """
    Create a BigQuery table with autodetect schema if it doesn't exist.
    
    Returns:
        The BigQuery table object
    """
    # Parse the table ID
    project, dataset, table_name = TABLE_ID.split('.')
    
    # Construct a full table reference
    table_ref = f"{project}.{dataset}.{table_name}"
    
    try:
        # Try to get the table
        table = client.get_table(table_ref)
        logger.info(f"Table {table_ref} already exists.")
        return table
    except Exception:
        # Table doesn't exist, so create it
        logger.info(f"Creating table {table_ref} with autodetect schema...")
        
        # Define the schema
        schema = [
            bigquery.SchemaField("Ticker", "STRING"),
            bigquery.SchemaField("Date", "DATE"),
            bigquery.SchemaField("Period", "STRING"),
            bigquery.SchemaField("Strong_Buy", "INTEGER"),
            bigquery.SchemaField("Buy", "INTEGER"),
            bigquery.SchemaField("Hold", "INTEGER"),
            bigquery.SchemaField("Sell", "INTEGER"),
            bigquery.SchemaField("Strong_Sell", "INTEGER"),
            bigquery.SchemaField("Target_Low", "FLOAT"),
            bigquery.SchemaField("Target_Mean", "FLOAT"),
            bigquery.SchemaField("Target_High", "FLOAT")
        ]
        
        # Create the table
        table = bigquery.Table(table_ref, schema=schema)
        
        # Set table creation options
        table.time_partitioning = bigquery.TimePartitioning(
            type_=bigquery.TimePartitioningType.DAY,
            field="Date"  # Partition by the Date column
        )
        
        # Create the table
        table = client.create_table(table)
        logger.info(f"Table {table_ref} created successfully.")
        return table

def save_to_bigquery(data: Dict[str, Any]) -> None:
    """Save combined data to BigQuery."""
    logger.info(f"Saving data to BigQuery table {TABLE_ID}...")
    try:
        # Ensure the table exists
        table = create_or_get_table()
        
        # Insert the rows
        rows_to_insert = [data]
        errors = client.insert_rows_json(table, rows_to_insert)
        
        if not errors:
            logger.info("Data successfully saved to BigQuery.")
        else:
            logger.error("Errors occurred while saving to BigQuery: %s", errors)
    except Exception as e:
        logger.error(f"Error saving data to BigQuery: {e}")

@functions_framework.http
def stock_data_handler(request):
    """Cloud Function entry point."""
    logger.info("Starting stock data extraction process")
    
    try:
        # Determine input method
        if request.method == 'POST':
            # For POST requests, try to get JSON from request body
            request_json = request.get_json(silent=True) or {}
        else:
            # For GET requests or when no JSON is found, use default tickers
            request_json = {}
        
        # Use provided tickers or default
        tickers_to_process = request_json.get('tickers', DEFAULT_TICKERS)
        
        # Process each ticker
        results = []
        for ticker_info in tickers_to_process:
            ticker = ticker_info.get('ticker', 'AAPL').upper()
            ticker_id = ticker_info.get('ticker_id', '146')
            
            yahoo_data = fetch_latest_yahoo_recommendations(ticker)
            seeking_alpha_data = fetch_seeking_alpha_targets(ticker_id)
            
            if yahoo_data and seeking_alpha_data:
                combined_data = {**yahoo_data, **seeking_alpha_data}
                save_to_bigquery(combined_data)
                results.append(f"Processed {ticker}")
            else:
                results.append(f"Failed to process {ticker}")
        
        # Return success response
        return (json.dumps({
            "message": "Stock data extraction complete", 
            "results": results
        }), 200, {'Content-Type': 'application/json'})
    
    except Exception as e:
        logger.error(f"Unexpected error in stock_data_handler: {e}")
        return (json.dumps({
            "error": "An unexpected error occurred",
            "details": str(e)
        }), 500, {'Content-Type': 'application/json'})

# For local testing
if __name__ == "__main__":
    # You can add local testing logic here if needed
    pass
        


In [36]:
import requests
import yfinance as yf

class StockPriceTargetRetriever:
    def __init__(self, api_key=None):
        """
        Initialize the Stock Price Target Retriever
        
        :param api_key: API key for paid services (optional)
        """
        self.api_key = api_key
    
    def get_yahoo_finance_target(self, symbol):
        """
        Retrieve price targets and recommendations from Yahoo Finance
        
        :param symbol: Stock ticker symbol
        :return: Dictionary with recommendations and price targets
        """
        try:
            # Fetch the stock information
            stock = yf.Ticker(symbol)
            
            # Fetch analyst recommendations
            recommendations = stock.recommendations
            
            # Fetch analyst price targets
            info = stock.info
            
            # Extract price target information from stock info
            price_targets = {
                'current_price': info.get('currentPrice'),
                'target_high_price': info.get('targetHighPrice'),
                'target_low_price': info.get('targetLowPrice'),
                'target_mean_price': info.get('targetMeanPrice'),
                'target_median_price': info.get('targetMedianPrice')
            }
            
            return {
                'recommendations': recommendations,
                'price_targets': price_targets
            }
        except Exception as e:
            print(f"Error fetching Yahoo Finance data: {e}")
            return None
    
    def get_alpha_vantage_overview(self, symbol):
        """
        Retrieve stock overview from Alpha Vantage
        
        :param symbol: Stock ticker symbol
        :return: Dictionary of stock overview data
        """
        if not self.api_key:
            raise ValueError("Alpha Vantage requires an API key")
        
        url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={self.api_key}'
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"Error fetching data from Alpha Vantage: {e}")
            return None
    
    def get_financial_modeling_prep_target(self, symbol):
        """
        Retrieve price targets from Financial Modeling Prep
        
        :param symbol: Stock ticker symbol
        :return: List of price target data
        """
        if not self.api_key:
            raise ValueError("Financial Modeling Prep requires an API key")
        
        url = f'https://financialmodelingprep.com/api/v3/price-target?symbol={symbol}&apikey={self.api_key}'
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"Error fetching data from Financial Modeling Prep: {e}")
            return None

def main():
    # Initialize the retriever
    retriever = StockPriceTargetRetriever()
    
    # Retrieve price targets for Apple (AAPL)
    symbol = 'ASTS'
    
    # Yahoo Finance (completely free)
    yahoo_targets = retriever.get_yahoo_finance_target(symbol)
    
    # Print results with error handling
    if yahoo_targets:
        print("Yahoo Finance Targets:")
        print("Recommendations:")
        print(yahoo_targets.get('recommendations', 'No recommendations available'))
        print("\nPrice Targets:")
        price_targets = yahoo_targets.get('price_targets', {})
        for key, value in price_targets.items():
            print(f"{key.replace('_', ' ').title()}: {value}")
    else:
        print("Failed to retrieve stock information.")

if __name__ == '__main__':
    main()

# Important Notes:
# 1. This script requires yfinance library
# 2. Install dependencies: pip install yfinance requests
# 3. Be aware of potential rate limits or changes in Yahoo Finance's structure

# Troubleshooting:
# - Ensure you have the latest version of yfinance
# - Some stock symbols might not have complete information
# - Network connectivity can affect data retrieval
   

Yahoo Finance Targets:
Recommendations:
  period  strongBuy  buy  hold  sell  strongSell
0     0m          2    3     0     0           0
1    -1m          2    3     0     0           0
2    -2m          2    3     0     0           0
3    -3m          2    3     0     0           0

Price Targets:
Current Price: 25.645
Target High Price: 53.0
Target Low Price: 15.0
Target Mean Price: 35.94
Target Median Price: 36.0


In [5]:
import smtplib
from email.mime.text import MIMEText

SMTP_SERVER = "smtp.zoho.com"
SMTP_PORT = 587
EMAIL_ADDRESS = "trendsense@zohomail.com"
EMAIL_PASSWORD = "pZNUVbUid0tv"
USER_EMAIL_ADDRESS = "4064315613@vtext.com"

try:
    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        message_body = "Test email from Cloud Function."
        msg = MIMEText(message_body)
        msg['Subject'] = "Test Email"
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = USER_EMAIL_ADDRESS
        server.sendmail(EMAIL_ADDRESS, USER_EMAIL_ADDRESS, msg.as_string())
    print("Email sent successfully.")
except Exception as e:
    print(f"Error sending email: {e}")

Email sent successfully.


In [ ]:
import smtplib
from email.mime.text import MIMEText

SMTP_SERVER = "smtp.zoho.com"
SMTP_PORT = 587
EMAIL_ADDRESS = "trendsense@zohomail.com"
EMAIL_PASSWORD = "pZNUVbUid0tv"
VERIZON_SMS_GATEWAY = "trendsense@zohomail.com"

try:
    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.set_debuglevel(1)  # Enable debugging output
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)

        message_body = "Test SMS notification from Zoho sent at 12:31 from vtext."
        msg = MIMEText(message_body)
        msg['Subject'] = ""  # Optional; may be ignored by SMS gateway
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = VERIZON_SMS_GATEWAY

        server.sendmail(EMAIL_ADDRESS, VERIZON_SMS_GATEWAY, msg.as_string())
    print("Email sent successfully.")
except Exception as e:
    print(f"Error sending SMS: {e}")

 # Email configuration - Replace with your actual credentials
    SMTP_SERVER = "smtp.zoho.com"
    SMTP_PORT = 587
    EMAIL_ADDRESS = "trendsense@zohomail.com"  # Replace with your email
    EMAIL_PASSWORD = "pZNUVbUid0tv"  # Replace with your password
    USER_EMAIL_ADDRESS = "trendsense@zohomail.com"  # Replace with recipient email

send: 'ehlo DESKTOP-1KT4FIT.home\r\n'
reply: b'250-mx.zohomail.com Hello DESKTOP-1KT4FIT.home (syn-184-166-077-250.res.spectrum.com (184.166.77.250))\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250 SIZE 32505856\r\n'
reply: retcode (250); Msg: b'mx.zohomail.com Hello DESKTOP-1KT4FIT.home (syn-184-166-077-250.res.spectrum.com (184.166.77.250))\nSTARTTLS\nSIZE 32505856'
send: 'STARTTLS\r\n'
reply: b'220 Ready to start TLS.\r\n'
reply: retcode (220); Msg: b'Ready to start TLS.'
send: 'ehlo DESKTOP-1KT4FIT.home\r\n'
reply: b'250-mx.zohomail.com Hello DESKTOP-1KT4FIT.home (syn-184-166-077-250.res.spectrum.com (184.166.77.250))\r\n'
reply: b'250-AUTH LOGIN PLAIN\r\n'
reply: b'250 SIZE 32505856\r\n'
reply: retcode (250); Msg: b'mx.zohomail.com Hello DESKTOP-1KT4FIT.home (syn-184-166-077-250.res.spectrum.com (184.166.77.250))\nAUTH LOGIN PLAIN\nSIZE 32505856'
send: 'AUTH PLAIN AHRyZW5kc2Vuc2VAem9ob21haWwuY29tAHBaTlVWYlVpZDB0dg==\r\n'
reply: b'235 Authentication Successful\r\n'
reply: retcode (235

Email sent successfully.


reply: b'250 Message received\r\n'
reply: retcode (250); Msg: b'Message received'
data: (250, b'Message received')
send: 'QUIT\r\n'
reply: b'221 mx.zohomail.com closing connection\r\n'
reply: retcode (221); Msg: b'mx.zohomail.com closing connection'
